In [2]:
%load_ext tensorboard

In [3]:
import tensorflow as tf
import datetime

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 7s 1us/step


In [5]:
num_classes = 10
input_shape = (28, 28, 1)
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ])

In [6]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 47s 24ms/step - loss: 0.2050 - accuracy: 0.9377 - val_loss: 0.0594 - val_accuracy: 0.9820
Epoch 2/5
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0800 - accuracy: 0.9753 - val_loss: 0.0477 - val_accuracy: 0.9843
Epoch 3/5
1875/1875 [==============================] - 44s 23ms/step - loss: 0.0648 - accuracy: 0.9800 - val_loss: 0.0351 - val_accuracy: 0.9884
Epoch 4/5
1875/1875 [==============================] - 44s 24ms/step - loss: 0.0551 - accuracy: 0.9831 - val_loss: 0.0326 - val_accuracy: 0.9891
Epoch 5/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0490 - accuracy: 0.9849 - val_loss: 0.0296 - val_accuracy: 0.9894


In [7]:
%tensorboard --logdir logs

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [10]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [11]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [12]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [13]:
model = create_model()

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.2673547565937042, Accuracy: 91.8550033569336, Test Loss: 0.06738223880529404, Test Accuracy: 97.88999938964844
Epoch 2, Loss: 0.0931592807173729, Accuracy: 97.20333099365234, Test Loss: 0.053422968834638596, Test Accuracy: 98.3499984741211
Epoch 3, Loss: 0.0726349949836731, Accuracy: 97.65333557128906, Test Loss: 0.039221469312906265, Test Accuracy: 98.68000030517578
Epoch 4, Loss: 0.060986343771219254, Accuracy: 98.11833190917969, Test Loss: 0.03806738927960396, Test Accuracy: 98.72999572753906
Epoch 5, Loss: 0.054122209548950195, Accuracy: 98.31500244140625, Test Loss: 0.03194553405046463, Test Accuracy: 98.94999694824219


In [14]:
%tensorboard --logdir logs/gradient_tape

In [15]:
!tensorboard dev upload \
  --logdir logs \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

^C
